### Langchain RAG LLM

1. From this site: https://python.langchain.com/docs/tutorials/rag/
2. This is just for learning purposes.
3. RAG usage in here to get the LLM to use the documents we have embedded using the embedded models.
4. The embedded data is being stored inside the vector store (can either be from db or even use the vector store offered by langchain)

In [2]:
import os
import bs4
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

In [4]:
# chat model with llm
llm = init_chat_model("o3-mini", model_provider="openai")

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# short term memory
vector_store = InMemoryVectorStore(embeddings)

In [7]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [ ]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

# can also create user defined or custom prompt using PromptTemplate

## Agent AI

In [9]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [10]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [11]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************tp4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [12]:
response = graph.invoke({"question": "What is the usage of Tools in LLM?"})
print(response["answer"])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************tp4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
response = graph.invoke({"question": "Can we make custom tools or only use the prebuilt one from models offered?"})
print(response["answer"])